# 📜 **Finance Laws Retrieval-Augmented Generation (RAG) System**
### **Ironhack Data Science and Machine Learning Bootcamp**
📅 **Date:** February 5, 2025  
📅 **Submission Date:** February 7, 2025  
👩‍💻 **Authors:** Ginosca Alejandro Dávila & Natanael Santiago Morales  

---

## **📌 Introduction**
Retrieval-Augmented Generation (**RAG**) is an AI framework that enhances **information retrieval** by combining:
- A **vector database** to retrieve relevant documents.
- A **Large Language Model (LLM)** to generate contextualized responses.

This approach is widely used in **legal research, finance, customer support, and knowledge management**, improving accuracy and relevance in automated systems.

---

## **📌 Project Overview**
This project aims to build a **RAG system** focused on **finance laws and regulations**. By integrating **document retrieval** and **LLM-based generation**, the system will provide **accurate, contextualized answers** to legal and financial queries.

We will experiment with different **embedding models** (e.g., OpenAI’s `text-embedding-3-large` vs. `all-MiniLM-L6-v2`) and **vector databases** (e.g., ChromaDB) to optimize retrieval and response generation.

---

## **📂 Dataset Description**
- **Source:** A collection of **finance laws, regulations, and legal texts**.  
- **Format:** `.zip` file containing multiple `.pdf` files, each representing legal provisions, case law, or financial regulations.  

---

## **🎯 Goals**
✔ Develop a **RAG pipeline** specialized in finance law and legal text retrieval.  
✔ Compare **OpenAI embeddings** and **Sentence Transformers** for legal text retrieval.  
✔ Evaluate the system based on **retrieval accuracy** and **response relevance**.  
✔ *(Bonus)* Deploy the system as a **web application** using Streamlit or Flask.  
✔ *(Bonus)* Incorporate **legal case studies** and **financial reports** for richer context.  

---

🔹 **Let’s dive in and build our Finance RAG system! 🚀**


## **Data Selection**
We chose the **Financial Laws Collection**, a dataset containing **11 PDF documents** on **financial legislation in the European Union**.  

Initially, we considered the **Puerto Rico News Articles** dataset, but its **unstructured and unreadable format** made it unsuitable for this project.  

Each document in our chosen dataset varies in **length and content**, covering **legal provisions, case law, and financial regulations**.


## **Library Imports & Setup**

We begin by **importing and installing the necessary libraries** for the project. The core library used is **LangChain**, a framework designed to simplify the creation of applications that use **Large Language Models (LLMs)**.

### **Key Libraries Used**
- **`langchain_openai`** → Integrates OpenAI models via their SDK. Used for `OpenAIEmbeddings`.
- **`langchain_huggingface`** → Enables the use of **Hugging Face embeddings** within LangChain.
- **`langchain_chroma`** → Provides integration with **ChromaDB**, an AI-native vector database.
- **`langchain_community`** → Loads and processes PDF documents, splitting them into pages.
- **`langchain_text_splitters`** → Splits text into **manageable chunks** for retrieval-based processing.

Additionally, we use:
- **`enchant`** → A spell-checking library that helps correct and merge split words.
- **`openai`** → To connect with OpenAI's API for embeddings and response generation.

We also **load the OpenAI API key** and initialize the OpenAI client.


In [ ]:
# Import necessary libraries
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

import os
import enchant
import re

from dotenv import load_dotenv
from IPython.display import Markdown


# Set your OpenAI API key
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")


from openai import OpenAI

client = OpenAI(
  api_key= API_KEY
)

## **Chunking the Text for Efficient Retrieval**

After extracting text from the PDF documents, we need to **split the text into chunks** before storing them in a vector database. This ensures **efficient retrieval** and **accurate responses**.

### **Text Splitter Selection**
We evaluated two splitting strategies:
1. **`CharacterTextSplitter`** → Splits text into **fixed-length chunks** with controlled overlap.  
2. **`RecursiveCharacterTextSplitter`** → Dynamically splits text but **often produces excessively small chunks**, reducing context.

### **Why We Chose `CharacterTextSplitter`**
✔ **Preserves context** using an **overlap of 200 characters**.  
✔ **Prevents excessive fragmentation**, ensuring chunks retain **complete, meaningful information**.  
✔ **Offers predictable behavior**, making retrieval tuning easier.

### **Implementation Steps**
1. **Initialized an empty list (`all_docs[]`)** to store processed chunks.
2. **Created a text splitter** (`CharacterTextSplitter`) with `chunk_size=1000` and `chunk_overlap=200`.
3. **Looped through the PDF files**, loading each document and splitting them into **pages**.
4. **Further split each page into smaller chunks**, maintaining meaningful context.
5. **Stored the processed chunks in `all_docs[]`**, preparing them for embedding and retrieval.

The following code block implements this process.


In [ ]:
# Directory containing the PDF files
document_dir = "./data/Finance_laws/EurLex_v1"

# Initialize an empty list to hold all documents
all_docs = []

# Create a text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Loop through all PDF files in the directory
for filename in os.listdir(document_dir):
    # Checking that we only take pdf files
    if filename.endswith(".pdf"):
        # after getting the file name, we add it to the path
        file_path = os.path.join(document_dir, filename)
        # Load the document and split it into pages
        pages = PyPDFLoader(file_path).load_and_split()
        # Split the text content of each page into smaller chunks
        docs = text_splitter.split_documents(pages)
        # Add the smaller chunks to the all_docs list
        all_docs.extend(docs)

print(f"Number of documents: {len(all_docs)}")

Number of documents: 1561


## **Examining a Sample Chunk**

After loading and chunking all the documents, we now examine a **sample chunk** from `all_docs[]`.  

Each chunk is represented as a **LangChain `Document` object**, which contains:
- **Metadata** → Includes the **document source** (file path) and **page number**.
- **Page Content** → The actual **text extracted** from that chunk.

Below is an example of how a chunk is structured:

```python
[
    Document(
        metadata={
            'source': './data/Finance_laws/EurLex_v1/AMLD_EURLEX.pdf', # Path to the document
            'page': 0,                                                  # Page number (from the document)
            'page_label': '1'                                           # Page number assigned by the text splitter.
        },
        page_content=                                                   # Text from the chunk
            "Chunk contents"
    )
]


In [ ]:
all_docs[0]

Document(metadata={'source': './data/Finance_laws/EurLex_v1\\AMLD_EURLEX.pdf', 'page': 0, 'page_label': '1'}, page_content="DIRECTIVES \nDIRECTIVE (EU) 2015/849 OF THE EUR OPEAN P ARLIAMENT AND OF THE COUNC IL \nof 20 May 2015 \non the prev ention of the use of the f inancial sys tem for the pur poses of money launder ing or \nter ror is t f inancing, amending Regulation (EU) No 648/2012 of the European Parliament and of the \nCouncil, and repealing Directiv e 2005/60/EC of the European Parliament and of the Council and \nCommission Directiv e 2006/70/EC \n(T ext with EEA relevance) \nTHE EUR OPEAN P ARLIAMENT AND THE COUNCIL OF THE EUR OPEAN UNION, \nHa ving regar d to the T reaty on the Functioning of the European Union, and in par ticular Ar ticle 114 thereof, \nHa ving regar d to the proposal from the European Commission, \nAf ter transmission of the draf t legislative act to the national parliaments, \nHa ving regar d to the opinion of the European Central Bank (\n1\n), \nHa ving 

## **Evaluating the Quality of Chunks**

To assess our chunking strategy, we print **the first chunk** of `all_docs[]`.

### **Key Checks**
✔ **Does each chunk retain full sentences?**  
✔ **Are words improperly split or cut off?**  
✔ **Are there excessive line breaks or formatting errors?**  

Identifying **text anomalies** early ensures clean data for embedding and retrieval.


In [ ]:
print(all_docs[0].page_content)
display(Markdown(all_docs[0].page_content))

DIRECTIVES 
DIRECTIVE (EU) 2015/849 OF THE EUR OPEAN P ARLIAMENT AND OF THE COUNC IL 
of 20 May 2015 
on the prev ention of the use of the f inancial sys tem for the pur poses of money launder ing or 
ter ror is t f inancing, amending Regulation (EU) No 648/2012 of the European Parliament and of the 
Council, and repealing Directiv e 2005/60/EC of the European Parliament and of the Council and 
Commission Directiv e 2006/70/EC 
(T ext with EEA relevance) 
THE EUR OPEAN P ARLIAMENT AND THE COUNCIL OF THE EUR OPEAN UNION, 
Ha ving regar d to the T reaty on the Functioning of the European Union, and in par ticular Ar ticle 114 thereof, 
Ha ving regar d to the proposal from the European Commission, 
Af ter transmission of the draf t legislative act to the national parliaments, 
Ha ving regar d to the opinion of the European Central Bank (
1
), 
Ha ving regar d to the opinion of the European Economic and Social Committ ee (
2
), 
A cting in accordance with the ordinar y legislative procedur

DIRECTIVES 
DIRECTIVE (EU) 2015/849 OF THE EUR OPEAN P ARLIAMENT AND OF THE COUNC IL 
of 20 May 2015 
on the prev ention of the use of the f inancial sys tem for the pur poses of money launder ing or 
ter ror is t f inancing, amending Regulation (EU) No 648/2012 of the European Parliament and of the 
Council, and repealing Directiv e 2005/60/EC of the European Parliament and of the Council and 
Commission Directiv e 2006/70/EC 
(T ext with EEA relevance) 
THE EUR OPEAN P ARLIAMENT AND THE COUNCIL OF THE EUR OPEAN UNION, 
Ha ving regar d to the T reaty on the Functioning of the European Union, and in par ticular Ar ticle 114 thereof, 
Ha ving regar d to the proposal from the European Commission, 
Af ter transmission of the draf t legislative act to the national parliaments, 
Ha ving regar d to the opinion of the European Central Bank (
1
), 
Ha ving regar d to the opinion of the European Economic and Social Committ ee (
2
), 
A cting in accordance with the ordinar y legislative procedure (
3
), 
Whereas: 
(1)  Flo ws of illicit mone y can damage the integrit y , stability and reputation of the financial sector , and threaten the 
internal market of the U nion as well as intern ational development. Mone y launder ing, terror ism financing and 
org anised cr ime remain significant problems which should be addressed at U nion level. In addition to fur ther 
developing the cr iminal la w approach at Union level, target ed and propor tionat e prevention of the use of the 
financia l system f or the pur poses of money launder ing and terr or ist financing is indispensable and can produce 
compl ementar y results. 
(2)  The soundness, integr ity and stability of credit institutions and fi nancial institutions, and confidence in the 
financia l system as a whole could be ser iously jeopardised by the eff or ts of cr iminals and their associates to 
disguise the or igin of cr iminal proceeds or to c hannel lawful or illicit money f or terr or ist pur poses. In order to 
f acilitate their cr iminal activities, mone y launderers and financer s of te r ror ism could tr y to take advantage of the 
freedom of capital mo vements and the freedom to supply fi nancial ser vices whic h the U nion's integrat ed fi nancial 
area entails. Theref ore, cer tain coordinating measures are necessar y at U nion level. A t the same time, the 
objectives of prot ecting society from cr ime and protectin g the stability and integrit y of the Union's financia l 
system should be balanced ag ainst the need to create a regulatory en vironment that allo ws compan ies to gro w 
their businesses without incur r ing dispropor tionate compl iance costs. 
(3)  This Directive is the f our th directive to address the threat of mone y launder ing. Council Directive 91/308/EEC (
4
) 
defined mone y launder ing in term s of dr ugs off ences and imposed obligations solely on the financial sector . 
5.6.2015 L 141/73 Official Jour nal of the European Union EN     
(
1
) OJ C 166, 12.6.2013, p. 2. 
(
2
) OJ C 271, 19.9.2013, p. 31. 
(
3
) P osition of the European P arliament of 11 March 2014 (not yet published in the Offi cial Jour nal) and position of the Council at first 
reading of 20 Apr il 2015 (not yet published in the Offi cial Jour nal). P osition of the European P arliament of 20 Ma y 2015 (not yet 
published in the Official Jour nal). 
(
4
) Council Directive 91/308/EEC of 10 June 1991 on prevention of the use of the f inancial syst em f or the pur pose of money launder ing 
(OJ L 166, 28.6.1991, p. 77).

## **Text Cleaning & Preprocessing**

To ensure high-quality retrieval, we implemented a **text cleaning function** that:
- **Removes unnecessary line breaks**, replacing them with spaces.
- **Merges split words** that were incorrectly separated due to PDF formatting.
- **Ensures proper spacing and punctuation formatting** for readability.

### **How the Cleaning Function Works**
1. **Tokenizes text into individual words and symbols**.  
2. **Attempts to merge adjacent words** that were mistakenly split.  
3. **Verifies word validity** using the **`enchant`** dictionary.  
   - If the merged word is valid, it replaces the split parts.  
   - If the merged word is invalid, the function keeps the original split.  
4. **Maintains punctuation and structural integrity** to ensure clean, readable text.  

This process significantly **improves text consistency**, making it more suitable for retrieval and embedding.


In [ ]:
import enchant
import re

# Load the English dictionary
d = enchant.Dict("en_US")

# Function to correct split words
def correct_split_words(sentence):
    # Replace newlines with spaces
    sentence = sentence.replace("\n", " ")

    words = sentence.split()  # Tokenize the sentence
    i = 0
    corrected_sentence = []

    while i < len(words) - 1:
        merged_word = words[i] + words[i+1]  # Merge adjacent words
        stripped_merged_word = re.sub(r'[^a-zA-Z]', '', merged_word) #remove symbols from merged word using regex

        # check if stripped merged word is empty
        if stripped_merged_word == '':
            corrected_sentence.append(words[i])
            i += 1
        elif d.check(stripped_merged_word):  # Check if merged word is in the dictionary
            corrected_sentence.append(merged_word)  # Add corrected word
            i += 2  # Skip the next word since it's already merged
        else:
            corrected_sentence.append(words[i])  # Add unmerged word
            i += 1  # Move to the next word

    # Add the last word if it wasn't merged
    if i < len(words):
        corrected_sentence.append(words[i])

    corrected_sentence = " ".join(corrected_sentence)

    corrected_sentence = re.sub(r'([,.")(])(?=\D)', r'\1 ', corrected_sentence)
    corrected_sentence = re.sub(r'\s+([,.)])', r'\1', corrected_sentence)
    corrected_sentence = re.sub(r'\(\s+', '(', corrected_sentence)
    corrected_sentence = re.sub(r'\s+', ' ', corrected_sentence).strip()
    corrected_sentence = re.sub(r'(\d)([A-Za-z])', r'\1 \2', corrected_sentence)
    corrected_sentence = re.sub(r'([A-Za-z]{2,})(\d+)', r'\1 \2', corrected_sentence)
    corrected_sentence = re.sub(r'(\w)(\()(?!s\))', r'\1 \2', corrected_sentence)

    return corrected_sentence

## **Applying Text Cleaning to All Chunks**
Now that we have defined the **`correct_split_words()` function**, we apply it to **every chunk in `all_docs[]`**.

This ensures that:

✔ **All retrieved text is well-formatted** before embedding.  
✔ **Errors from extraction are minimized.**  
✔ **Retrieval quality is improved** since the model will process clearer text.


In [ ]:
# Apply the function to all documents in `all_docs[]`
for doc in all_docs:
    doc.page_content = correct_split_words(doc.page_content)

print(all_docs[0].page_content)

DIRECTIVES DIRECTIVE (EU)2015/849 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 20 May 2015 on the prevention of the use of the financial system for the purposes of money laundering or terror is t financing, amending Regulation (EU) No 648/2012 of the European Parliament and of the Council, and repealing Directive 2005/60/EC of the European Parliament and of the Council and Commission Directive 2006/70/EC (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Ha ving regard to the opinion of the European Economic and Social Committee (2), A cting in accordance with the ordinary legislative procedure (3), Whereas: (1) Flo ws of illicit money can d

## **Verifying Cleaned Text**
We now print the **same chunk as before** after applying text cleaning.

### **Improvements Observed**
✔ **No more unwanted line breaks**.  
✔ **Words that were incorrectly split are now properly merged**.  
✔ **The text is more structured and readable**.  

This cleaned version will be **embedded into our vector database** for retrieval.


In [ ]:
print(all_docs[0].page_content)
display(Markdown(all_docs[0].page_content))

DIRECTIVES DIRECTIVE (EU)2015/849 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 20 May 2015 on the prevention of the use of the financial system for the purposes of money laundering or terror is t financing, amending Regulation (EU) No 648/2012 of the European Parliament and of the Council, and repealing Directive 2005/60/EC of the European Parliament and of the Council and Commission Directive 2006/70/EC (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Ha ving regard to the opinion of the European Economic and Social Committee (2), A cting in accordance with the ordinary legislative procedure (3), Whereas: (1) Flo ws of illicit money can d

DIRECTIVES DIRECTIVE (EU)2015/849 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 20 May 2015 on the prevention of the use of the financial system for the purposes of money laundering or terror is t financing, amending Regulation (EU) No 648/2012 of the European Parliament and of the Council, and repealing Directive 2005/60/EC of the European Parliament and of the Council and Commission Directive 2006/70/EC (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Ha ving regard to the opinion of the European Economic and Social Committee (2), A cting in accordance with the ordinary legislative procedure (3), Whereas: (1) Flo ws of illicit money can damage the integrity, stability and reputation of the financial sector, and threaten the internal market of the Union as well as international development. Money laundering, terrorism financing and org anised crime remain significant problems which should be addressed at Union level. In addition to further developing the criminal law approach at Union level, targeted and propor tionat e prevention of the use of the financial system for the purposes of money laundering and terror ist financing is indispensable and can produce compl ementar y results. (2) The soundness, integrity and stability of credit institutions and financial institutions, and confidence in the financial system as a whole could be seriously jeopardised by the eff orts of criminals and their associates to disguise the origin of criminal proceeds or to channel lawful or illicit money for terror ist purposes. In order to facilitate their criminal activities, money launderers and financer s of ter ror ism could try to take advantage of the freedom of capital movements and the freedom to supply financial services which the Union's integrated financial area entails. Therefore, certain coordinating measures are necessary at Union level. At the same time, the objectives of protecting society from crime and protecting the stability and integrity of the Union's financial system should be balanced against the need to create a regulatory environment that allows companies to grow their businesses without incur ring disproportionate compliance costs. (3) This Directive is the four th directive to address the threat of money laundering. Council Directive 91/308/EEC (4) defined money laundering in terms of drugs off ences and imposed obligations solely on the financial sector. 5.6.2015 L 141/73 Official Journal of the European Union EN (1) OJ C166, 12.6.2013, p. 2. (2) OJ C271, 19.9.2013, p. 31. (3) P osition of the European Parliament of 11 March 2014 (not yet published in the Official Journal) and position of the Council at first reading of 20 April 2015 (not yet published in the Official Journal). Position of the European Parliament of 20 May 2015 (not yet published in the Official Journal). (4) Council Directive 91/308/EEC of 10 June 1991 on prevention of the use of the financial system for the purpose of money laundering (OJ L166, 28.6.1991, p. 77).

## **Embedding the Text for Retrieval**
We generate **vector embeddings** for each document chunk using **two different models**:

1. **OpenAI Embeddings (`text-embedding-3-large`)**  
   - Commercial model known for **high accuracy and performance**.  
2. **Hugging Face Embeddings (`all-MiniLM-L6-v2`)**  
   - Open-source alternative that balances **efficiency and quality**.

Using **two different embedding methods** allows us to compare performance and optimize retrieval accuracy.


In [ ]:
# Create the embeddings function

openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large", api_key=API_KEY)
print("Initialized OpenAI embeddings.")

sentence_transformer_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Initialized SentenceTransformer embeddings.")

Initialized OpenAI embeddings.

Initialized SentenceTransformer embeddings.


## **Storing the Chunks in the Vector Database**

After initializing the embeddings, we created a vector database using **ChromaDB** and stored the embeddings of each chunk in the database. 

We created **two databases**: 
1. One for the **OpenAI** embeddings and 
2. One for the **Hugging Face** embeddings.

In [ ]:
# Load all documents into Chroma
db = Chroma.from_documents(all_docs, sentence_transformer_embeddings, persist_directory="./chroma_db_clean_huggingface")
print('All documents loaded and embedded.(huggingface)')

db_openai = Chroma.from_documents(all_docs, openai_embeddings, persist_directory="./chroma_db_clean_openai")
print('All documents loaded and embedded.(openai)')

All documents loaded and embedded.(huggingface)
All documents loaded and embedded.(openai)


## **Retrieving Similar Chunks**
We use **vector similarity search** to find the most relevant document chunks for a given query.

### **Retrieval Process**
1. **Convert the user query into an embedding**.
2. **Compare it against stored document embeddings**.
3. **Retrieve the top `k=5` most similar chunks**.

Below, we retrieve **3 most relevant chunks** using:
- **Hugging Face embeddings** (`all-MiniLM-L6-v2`).


In [ ]:
# Example query
user_question = "What are the key points of financial legislation in Europe?"
docs = db.similarity_search(user_question, k=5)

# Print results
for doc in docs[0:3]:
    # print(doc.page_content, '\n')
    display(Markdown(doc.page_content))
    print("*"*80, '\n')

I (Legislative acts) REGULATIONS REGULATION (EU) No 575/2013 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 26 June 2013 on prudential requirements for credit institutions and investment firms and amending Regulation (EU) No 648/2012 (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Having regard to the opinion of the European Economic and Social Committee (2), Acting in accordance with the ordinary legislative procedure, Whereas:(1) The G-20 Declaration of 2 April 2009 on Strengthening of the Financial System called for internationally consistent efforts that are aimed at strengthening trans­ parency, accountability and regulation by improving the quantity and quality of capital in the banking system once the economic recovery is assured. That declaration also called for introduction of a supplementary non-risk based measure to contain the build-up of leverage in the banking system, and the development of a framework for stronger liquidity buffers. In response to the mandate given by the G-20, in September 2009 the Group of Central Bank Governors and Heads of Supervision (GHOS), agreed on a number of measures to strengthen the regulation of the banking sector. Those measures were endorsed by the G-20 leaders at their Pittsburgh Summit of 24-25 September 2009 and were set out in detail in December 2009. In July and September 2010, GHOS issued two further announcements on design and calibration of those new measures, and in December 2010, the Basel Committee on Banking Supervision (BCBS) published the final measures, that are referred to as the Basel III framework. (2) The High Level Group on Financial Supervision in the EU chaired by Jacques de Larosière (the " de Larosière group") invited the Union to develop a more harmonised set of financial regulations. In the context of the future European supervisory architecture, the European Council of 18 and 19 June 2009 also stressed the need to establish a' European Single Rule Book' applicable to all credit institutions and investment firms in the internal market. (3) As stated in the de Larosière group's report of 25 February 2009 (the" de Larosière report"), " a Member State should be able to adopt more stringent national regulatory measures considered to be domes­ tically appropriate for safeguarding financial stability as long as the principles of the internal market and agreed minimum core standards are respected". (4) Directive 2006/48/EC of the European Parliament and of the Council of 14 June 2006 relating to the taking up and pursuit of the business of credit institutions (3) and Directive 2006/49/EC of the European Parliament and of EN 27.6.2013 Official Journal of the European Union L176/1 (1) OJ C105, 11.4.2012, p. 1. (2) OJ C68, 6.3.2012, p. 39. (3) OJ L177, 30.6.2006, p. 1.

******************************************************************************** 



(121) In order to take account of market developments and experience in the application of this Regulation, the Commission should be required to submit reports to the European Parliament and to the Council, together with legislative proposals, where appropriate, on the possible effect of capital requirements on the economic cycle of minimum, own funds requirements for exposures in the form of covered bonds, large exposures, liquidity requirements, leverage, exposures to transferred credit risk, counterparty credit risk and the original exposure method, retail exposures, on the definition of eligible capital, and the level of application of this Regu ­ lation. (122) The primary purpose of the legal framework for credit institutions should be to ensure the operation of vital services to the real economy while limiting the risk of moral hazard. The structural separation of retail and investment banking activities within a banking group could be one of the key tools to support this objective. No provision in the current regulatory framework should therefore prevent the introduction of measures to effect such a separation. The Commission should be required to analyse the issue of structural separation in the Union and submit a report, together with legislative proposals, if appropriate, to the European Parliament and the Council. (123) Similarly, with a view to protecting depositors and preserving financial stability, Member States should also be permitted to adopt structural measures that require credit institutions authorised in that Member State to reduce their exposures to different legal entities depending on their activities, irrespective of where those activities are located. However, because such measures could have a negative impact by fragmenting the internal market, they should only be approved subject to strict conditions pending the entry into force of a future legal act explicitly harmonising such measures. (124) In order to specify the requirements set out in this Regu ­ lation, the power to adopt acts in accordance with Article 290 TFEU should be delegated to the Commission in respect of technical adjustments to this Regulation to clarify definitions to ensure uniform appli ­cation of this Regulation or to take account of devel ­ opments on financial markets, to align terminology on, and frame definitions in accordance with, subsequent relevant acts, to adjust the provisions of this Regulation on own funds to reflect developments in accounting standards or Union law, or with regard to the convergence of supervisory practices, to expand the lists of exposure classes for the purposes of the Stan­ dardised Approach or the IRB Approach to take account of developments on financial markets, to adjust certain amounts relevant to those exposure classes to take into account the effects of inflation; to adjust the list and classification of off- balance sheet items and to adjust specific provisions and technical criteria on the treatment of counterparty credit risk, the Standardised Approach and the IRB Approach, credit risk mitigation, securitisation, operational risk, market risk, liquidity, leverage and disclosure in order to take account of developments on financial markets or in accounting standards or Union law, or with regard to the convergence of supervisory practices and risk measurement and to take account of the outcome of the review of various matters relating to the scope of Directive 2004/39/EC. (125) The power to adopt acts in accordance with Article 290 TFEU should also be delegated to the Commission in respect of prescribing a temporary reduction in the level of own funds or risk weights specified under this Regulation in order to take account of specific circum ­stances, to clarify the exemption of certain exposures from the application of provisions of this Regulation on large exposures, to specify amounts relevant to the

******************************************************************************** 



(4) Directive 2004/39/EC of the European Parliament and of the Council of 21 April 2004 on markets in financial instruments (1) allows investment firms authorised by the competent authorities of their home Member State and supervised by the same authorities to establish branches and provide services freely in other Member States. That Directive accordingly provides for the coordination of the rules governing the authorisation and pursuit of the business of investment firms. It does not, however, establish the amounts of the initial capital of such firms ora common framework for monitoring the risks incurred by them, which should be provided by this Directive. (5) This Directive should constitute the essential instrument for the achievement of the internal market from the point of view of both the freedom of establishment and the freedom to provide financial services in the field of credit institutions. (6) The smooth operation of the internal market requires not only legal rules but also close and regular cooperation and significantly enhanced convergence of regulatory and supervisory practices between the competent authorities of the Member States. (7) Regulation (EU) No 1093/2010 of the European Parliament and of the Council (2) established the European Supervisory Authority (European Banking Authority) (" EBA"). This Directive should take into account the role and function of EBA set out in that Regulation and the procedures to be followed when conferring tasks on EBA. (8) Given the increase of tasks conferred on EBA by this Directive and by Regulation (EU) No 575/2013, the European Parliament, the Council and the Commission should ensure that adequate human and financial resources are made available. (9) Asa first step towards a banking union, a single super­ visory mechanism (SSM) should ensure that the Union's policy relating to the prudential supervision of credit institutions is implemented in a coherent and effective manner, that the single rulebook for financial services is applied in the same manner to credit institutions in all Member States concerned, and that those credit insti ­ tutions are subject to supervision of the highest quality, unfettered by other, non-prudential considerations. An SSM is the basis for the next steps towards a banking union. This reflects the principle that any introduction of common intervention mechanisms in the event of a crisis should be preceded by common controls to reduce the likelihood that such intervention mechanisms will have to be used. The European Council noted in its conclusions of 14 December 2012 that " the Commission will submit in the course of 2013 a proposal fora single resolution mechanism for Member States participating in the SSM, to be examined by co-legislators as a matter of priority with the intention of adopting it during the current parliamentary cycle. ". The integration of the financial framework could be further enhanced through the setting up of a single resolution mechanism, including appropriate and effective backstop arrangements to ensure that bank resolution decisions are taken swiftly, impartially and in the best interests of all concerned. (10) The conferral of supervisory tasks on the European Central Bank (ECB) for some of the Member States should be consistent with the framework of the European System of Financial Supervision setup in 2010 and its underlying objective to develop the single rulebook and enhance convergence of supervisory practices across the Union as a whole. The ECB should carryout its tasks subject to and in compliance with any relevant primary and secondary Union law, Commission decisions in the areas of State aid, competition rules and merger control and the single rulebook applying to all Member States. EBA is entrusted with developing draft technical standards and guidelines and recommendations

******************************************************************************** 



## **Retrieving Similar Chunks**
We use **vector similarity search** to find document chunks most relevant to a given query.

### **Retrieval Process**
1. **Convert the user query into an embedding**.
2. **Compare it against stored document embeddings**.
3. **Retrieve the top `k=5` most similar chunks**.

Below, we retrieve **3 most relevant chunks** using:
- **OpenAI embeddings** (`text-embedding-3-large`).


In [ ]:
# Example query
user_question = "What are the key points of financial legislation in Europe?"
docs_openai = db.similarity_search(user_question, k=5)

# Print results
for doc in docs_openai[0:3]:
    # print(doc.page_content, '\n')
    display(Markdown(doc.page_content))
    print("*"*80, '\n')

I (Legislative acts) REGULATIONS REGULATION (EU) No 575/2013 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL of 26 June 2013 on prudential requirements for credit institutions and investment firms and amending Regulation (EU) No 648/2012 (Text with EEA relevance) THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION, Having regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof, Having regard to the proposal from the European Commission, After transmission of the draft legislative act to the national parliaments, Having regard to the opinion of the European Central Bank (1), Having regard to the opinion of the European Economic and Social Committee (2), Acting in accordance with the ordinary legislative procedure, Whereas:(1) The G-20 Declaration of 2 April 2009 on Strengthening of the Financial System called for internationally consistent efforts that are aimed at strengthening trans­ parency, accountability and regulation by improving the quantity and quality of capital in the banking system once the economic recovery is assured. That declaration also called for introduction of a supplementary non-risk based measure to contain the build-up of leverage in the banking system, and the development of a framework for stronger liquidity buffers. In response to the mandate given by the G-20, in September 2009 the Group of Central Bank Governors and Heads of Supervision (GHOS), agreed on a number of measures to strengthen the regulation of the banking sector. Those measures were endorsed by the G-20 leaders at their Pittsburgh Summit of 24-25 September 2009 and were set out in detail in December 2009. In July and September 2010, GHOS issued two further announcements on design and calibration of those new measures, and in December 2010, the Basel Committee on Banking Supervision (BCBS) published the final measures, that are referred to as the Basel III framework. (2) The High Level Group on Financial Supervision in the EU chaired by Jacques de Larosière (the " de Larosière group") invited the Union to develop a more harmonised set of financial regulations. In the context of the future European supervisory architecture, the European Council of 18 and 19 June 2009 also stressed the need to establish a' European Single Rule Book' applicable to all credit institutions and investment firms in the internal market. (3) As stated in the de Larosière group's report of 25 February 2009 (the" de Larosière report"), " a Member State should be able to adopt more stringent national regulatory measures considered to be domes­ tically appropriate for safeguarding financial stability as long as the principles of the internal market and agreed minimum core standards are respected". (4) Directive 2006/48/EC of the European Parliament and of the Council of 14 June 2006 relating to the taking up and pursuit of the business of credit institutions (3) and Directive 2006/49/EC of the European Parliament and of EN 27.6.2013 Official Journal of the European Union L176/1 (1) OJ C105, 11.4.2012, p. 1. (2) OJ C68, 6.3.2012, p. 39. (3) OJ L177, 30.6.2006, p. 1.

******************************************************************************** 



(121) In order to take account of market developments and experience in the application of this Regulation, the Commission should be required to submit reports to the European Parliament and to the Council, together with legislative proposals, where appropriate, on the possible effect of capital requirements on the economic cycle of minimum, own funds requirements for exposures in the form of covered bonds, large exposures, liquidity requirements, leverage, exposures to transferred credit risk, counterparty credit risk and the original exposure method, retail exposures, on the definition of eligible capital, and the level of application of this Regu ­ lation. (122) The primary purpose of the legal framework for credit institutions should be to ensure the operation of vital services to the real economy while limiting the risk of moral hazard. The structural separation of retail and investment banking activities within a banking group could be one of the key tools to support this objective. No provision in the current regulatory framework should therefore prevent the introduction of measures to effect such a separation. The Commission should be required to analyse the issue of structural separation in the Union and submit a report, together with legislative proposals, if appropriate, to the European Parliament and the Council. (123) Similarly, with a view to protecting depositors and preserving financial stability, Member States should also be permitted to adopt structural measures that require credit institutions authorised in that Member State to reduce their exposures to different legal entities depending on their activities, irrespective of where those activities are located. However, because such measures could have a negative impact by fragmenting the internal market, they should only be approved subject to strict conditions pending the entry into force of a future legal act explicitly harmonising such measures. (124) In order to specify the requirements set out in this Regu ­ lation, the power to adopt acts in accordance with Article 290 TFEU should be delegated to the Commission in respect of technical adjustments to this Regulation to clarify definitions to ensure uniform appli ­cation of this Regulation or to take account of devel ­ opments on financial markets, to align terminology on, and frame definitions in accordance with, subsequent relevant acts, to adjust the provisions of this Regulation on own funds to reflect developments in accounting standards or Union law, or with regard to the convergence of supervisory practices, to expand the lists of exposure classes for the purposes of the Stan­ dardised Approach or the IRB Approach to take account of developments on financial markets, to adjust certain amounts relevant to those exposure classes to take into account the effects of inflation; to adjust the list and classification of off- balance sheet items and to adjust specific provisions and technical criteria on the treatment of counterparty credit risk, the Standardised Approach and the IRB Approach, credit risk mitigation, securitisation, operational risk, market risk, liquidity, leverage and disclosure in order to take account of developments on financial markets or in accounting standards or Union law, or with regard to the convergence of supervisory practices and risk measurement and to take account of the outcome of the review of various matters relating to the scope of Directive 2004/39/EC. (125) The power to adopt acts in accordance with Article 290 TFEU should also be delegated to the Commission in respect of prescribing a temporary reduction in the level of own funds or risk weights specified under this Regulation in order to take account of specific circum ­stances, to clarify the exemption of certain exposures from the application of provisions of this Regulation on large exposures, to specify amounts relevant to the

******************************************************************************** 



(4) Directive 2004/39/EC of the European Parliament and of the Council of 21 April 2004 on markets in financial instruments (1) allows investment firms authorised by the competent authorities of their home Member State and supervised by the same authorities to establish branches and provide services freely in other Member States. That Directive accordingly provides for the coordination of the rules governing the authorisation and pursuit of the business of investment firms. It does not, however, establish the amounts of the initial capital of such firms ora common framework for monitoring the risks incurred by them, which should be provided by this Directive. (5) This Directive should constitute the essential instrument for the achievement of the internal market from the point of view of both the freedom of establishment and the freedom to provide financial services in the field of credit institutions. (6) The smooth operation of the internal market requires not only legal rules but also close and regular cooperation and significantly enhanced convergence of regulatory and supervisory practices between the competent authorities of the Member States. (7) Regulation (EU) No 1093/2010 of the European Parliament and of the Council (2) established the European Supervisory Authority (European Banking Authority) (" EBA"). This Directive should take into account the role and function of EBA set out in that Regulation and the procedures to be followed when conferring tasks on EBA. (8) Given the increase of tasks conferred on EBA by this Directive and by Regulation (EU) No 575/2013, the European Parliament, the Council and the Commission should ensure that adequate human and financial resources are made available. (9) Asa first step towards a banking union, a single super­ visory mechanism (SSM) should ensure that the Union's policy relating to the prudential supervision of credit institutions is implemented in a coherent and effective manner, that the single rulebook for financial services is applied in the same manner to credit institutions in all Member States concerned, and that those credit insti ­ tutions are subject to supervision of the highest quality, unfettered by other, non-prudential considerations. An SSM is the basis for the next steps towards a banking union. This reflects the principle that any introduction of common intervention mechanisms in the event of a crisis should be preceded by common controls to reduce the likelihood that such intervention mechanisms will have to be used. The European Council noted in its conclusions of 14 December 2012 that " the Commission will submit in the course of 2013 a proposal fora single resolution mechanism for Member States participating in the SSM, to be examined by co-legislators as a matter of priority with the intention of adopting it during the current parliamentary cycle. ". The integration of the financial framework could be further enhanced through the setting up of a single resolution mechanism, including appropriate and effective backstop arrangements to ensure that bank resolution decisions are taken swiftly, impartially and in the best interests of all concerned. (10) The conferral of supervisory tasks on the European Central Bank (ECB) for some of the Member States should be consistent with the framework of the European System of Financial Supervision setup in 2010 and its underlying objective to develop the single rulebook and enhance convergence of supervisory practices across the Union as a whole. The ECB should carryout its tasks subject to and in compliance with any relevant primary and secondary Union law, Commission decisions in the areas of State aid, competition rules and merger control and the single rulebook applying to all Member States. EBA is entrusted with developing draft technical standards and guidelines and recommendations

******************************************************************************** 



## **Loading the Databases**

To efficiently retrieve relevant legal documents, we load two separate databases, each utilizing a different embedding model. This allows for flexibility in selecting the appropriate model for different use cases.

### **Database Loading Process**
1. **Hugging Face Embeddings Database**: 
   - The first database leverages **Hugging Face embeddings** for similarity search.
   - We load it from the specified persistence directory: `./chroma_db_clean_huggingface`.

2. **OpenAI Embeddings Database**: 
   - The second database uses **OpenAI embeddings** for similarity search.
   - It is loaded from the directory: `./chroma_db_clean_openai`.


In [ ]:
print("Loading the databases")

persist_directory_huggingface = "./chroma_db_clean_huggingface"
db = Chroma(persist_directory=persist_directory_huggingface, embedding_function=sentence_transformer_embeddings)
print("Huggingface database loaded.")

persist_directory_openai = "./chroma_db_clean_openai"
db_openai = Chroma(persist_directory=persist_directory_openai, embedding_function=openai_embeddings)
print("OpenAI database loaded.")

## **Formatting Retrieved Chunks for Prompting**
To **generate responses**, we need to format the retrieved chunks into a structured prompt.

The `_get_document_prompt()` function:
1. Extracts **metadata** (source file, page number).
2. Combines **relevant document chunks** into a single text block.
3. Structures the information **so that the LLM can generate more accurate answers**.

This ensures that responses are **contextually accurate and properly cited**.


In [ ]:
# Function to combine documents into a single string
def _get_document_prompt(docs):
    """
    Combine the content of multiple documents into a single string

    Args:
    docs (list): A list of Document objects

    Returns:
    str: A string containing the content of all documents
    """

    prompt = ""
    for doc in docs:
        # Extract metadata for citation
        source = doc.metadata.get('source', 'Unknown source')
        page = doc.metadata.get('page', 'Unknown page')

        # Include both metadata and content in the prompt
        prompt += f"Source: {source}, Page: {page}\n{doc.page_content}\n\n"

    return prompt


To **provide context for answering user questions**, the `get_context()` function retrieves relevant document chunks based on the query:

1. **Arguments**:
    - `user_question`: The **query** posed by the user.
    - `k`: The number of **top documents** to retrieve.
    - `openai`: A **flag** indicating whether to use **OpenAI** embeddings or **SentenceTransformer** embeddings.

2. **Logic**:
    - Depending on the **embedding model**, the function retrieves the most **similar documents** using Chroma.
    - The function then **formats the retrieved content** using `_get_document_prompt()` to provide **context for the response**.

This ensures that the generated answer is based on the most relevant chunks of information.


In [ ]:
def get_context (user_question, k, openai):
    """
    Get the context for the user question

    Args:
    user_question (str): The user question
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings

    Returns:
    str: The context for the user question
    """
    
    if openai:
        docs = db_openai.similarity_search(user_question, k)
    else:
        docs = db.similarity_search(user_question, k)
    return _get_document_prompt(docs)

The `get_prompt()` function is designed to create a **structured prompt** for answering legal questions based on **European Union Financial Laws**.

1. **Arguments**:
    - `user_question`: The **user’s query**.
    - `k`: The number of **documents to retrieve** (default: 10).
    - `openai`: A flag for using either **OpenAI** embeddings or **SentenceTransformer** embeddings.

2. **Logic**:
    - The function generates a **prompt** with the following sections:
      - **Introduction**: Clarifies the chatbot’s role as a legal assistant.
      - **Context**: Provides relevant legal documentation, retrieved using `get_context()`.
      - **Restrictions**: Outlines the chatbot’s behavioral **constraints** (e.g., only factual responses).
      - **Task**: Specifies the **answering structure** and the **format** for responding.

3. **Output**:
    - A **well-formatted prompt** containing the legal context, guidelines for responding, and a structure for the response, including **citation** of the source documents and pages.

This ensures that the chatbot gives precise, legally accurate, and properly cited answers.


In [ ]:
def get_prompt (user_question, k=10, openai=False):
    """
    Generate a prompt for the user question

    Args:
    user_question (str): The user question
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings

    Returns:
    str: The prompt for the user question
    """

    return f"""
## INTRODUCTION
You are a Chatbot designed to help lawyers with European Union Financial Laws. 
The user asked: "{user_question}"

## CONTEXT
Legal Documentation related to the users' question is provided below:
'''
{get_context(user_question, k, openai)}
'''

## RESTRICTIONS
Refer to the laws by their name, number, or article. 
Be clear, transparent, and factual: only state what is in the context without providing opinions or subjectivity.
Answer the question based solely on the context above; if you do not know the answer, be clear with the user that you do not know.
Only respond to questions related to the laws, avoiding jokes, offensive remarks, and discussions on religion or sexuality, unless discussed in the context.
If the user does not provide sufficient context, do not answer and instead ask for more information on what the user wants to know.

## TASK
First, answer directly to the user, if possible.
Second, reference the source of the information you provided in parentheses, next to the answer.
Lastly, answer in Markdown format.

## RESPONSE STRUCTURE:
'''
# [Answer Title]
[answer text (reference). Keep answering and (reference) until you have answered all parts of the question.]

Source:
- From `[Document name], page [page number]` of the European Union Finance Laws, from the Official Journal of the European Union
- From `[Document name], page [page number]` of the European Union Finance Laws, from the Official Journal of the European Union
'''
Ignore the square brackets and replace the text inside with the relevant information. For example, if the metadata of the used document is "Source: './data/Finance_laws/EurLex_v1/AMLD_EURLEX.pdf', Page: 0", the source would be 'AMLD_EURLEX.pdf' and the page number would be '0'. 

## CONVERSATION:
User: {user_question}
Agent:

"""

The `get_message()` function creates a **message object** to send to the chatbot API.

1. **Arguments**:
    - `user_question`: The **query** from the user.
    - `k`: The number of **documents to retrieve** for context (default: 10).
    - `openai`: Boolean flag indicating whether to use **OpenAI** or **SentenceTransformer** embeddings.

2. **Logic**:
    - This function calls `get_prompt()` to generate a **full prompt** (question, context, etc.).
    - It then formats the prompt into a **list** with a single message:
      - `role`: Defines the message as coming from the **user**.
      - `content`: Contains the **formatted prompt**.

3. **Output**:
    - A list containing the **message object** that can be passed to the chatbot API for processing.

This ensures that the chatbot receives the user’s question in a structured and clear format, along with the required legal context.


In [ ]:
def get_message(user_question, k=10, openai=False):
    """
    Get the initial message for the user question
    
    Args:
    user_question (str): The user question
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings
    
    Returns:
    list: A list containing the initial message
    """
    
    return [{'role': 'user', 'content': get_prompt(user_question, k, openai)}]

## **Generating Responses with RAG**
Now that we have:

✔ **Loaded and embedded the documents**  
✔ **Implemented vector similarity search**  
✔ **Formatted retrieved chunks into a structured prompt**  

We can now use **GPT-4o-mini** to **generate answers** based on the retrieved legal documents.


In [ ]:
def get_response(user_question, just_message = True, model = "gpt-4o-mini", k=10, openai = False, **kwargs):
    """
    Get the response to the user question
    
    Args:
    user_question (str): The user question
    just_message (bool): Whether to return the message only or the whole object
    model (str): The model to use for the response
    k (int): The number of documents to retrieve
    openai (bool): Whether to use OpenAI or SentenceTransformer embeddings
    **kwargs: Additional keyword arguments for the completion, such as `temperature`, `max_tokens`, etc.
    
    Returns:
    str: The response to the user question
    """
    
    message = get_message(user_question, k, openai)

    completion = client.chat.completions.create(
        model=model,
        messages=message,
        **kwargs
    )
    
    if just_message:
        return completion.choices[0].message.content
    
    return completion.choices[0]

To test the chatbot's capabilities, a **legal question** was derived from a specific section of the document.

- **Source**: `data/Finance_laws/EurLex_v1/BRRD_EURLEX.pdf`
- **Location**: Page 40, Section 3, Article 10, Paragraph 8

The text outlines the **responsibilities of Member States** in ensuring that resolution authorities have the power to request financial records from institutions. The specific clause mentions the ability of the resolution authority to set time limits for producing these records, with the possibility of different limits for various contract types.

### Example Text:
> Member States shall ensure that resolution authorities have the power to require an institution and an entity referred to in point (b), (c) or (d) of Article 1(1) to maintain detailed records of financial contracts to which it is a party...

From this **paragraph**, we can form a question to ask the chatbot, such as:
- *"Do authorities have the power to require an institution to mantain records of financial contracts?"*

This demonstrates how **specific sections** of the document are extracted and **transformed into relevant legal questions** for the chatbot to process.


## **Hugging Face Embeddings Response**

We use **Hugging Face embeddings** to answer legal questions by retrieving the most relevant document chunks from the database. This process helps ensure that responses are accurate and grounded in legal documents.

### **Response Process**
1. **Convert the user's legal question** into an embedding using Hugging Face models.
2. **Perform a similarity search** to retrieve the most relevant legal text from the database.
3. **Generate a response** using the retrieved text and structure it appropriately.


In [ ]:
question = "Do authorities have the power to require an institution to mantain records of financial contracts?"

response = get_response(question)

display(Markdown(response))

# Authority to Require Record-Keeping
Yes, authorities do have the power to require an institution to maintain detailed records of financial contracts to which it is a party. Member States are mandated to ensure that resolution authorities possess this power and may specify time-limits for institutions to be capable of producing those records. The resolution authority can also set different time-limits for different types of financial contracts (BRRD, Article 45(1), page 40).

Source:
- From `BRRD_EURLEX.pdf`, page 40 of the European Union Finance Laws, from the Official Journal of the European Union.

## **OpenAI Embeddings Response**

We use **OpenAI embeddings** to handle legal questions by performing a similarity search to find the most relevant chunks of legal text from the database. This method leverages powerful models like **`text-embedding-3-large`** to retrieve and structure answers.

### **Response Process**
1. **Convert the user's legal question** into an embedding using OpenAI models.
2. **Perform a similarity search** to retrieve the most relevant legal text.
3. **Generate a response** based on the retrieved text and present it clearly.


In [ ]:
question = "Do authorities have the power to require an institution to mantain records of financial contracts?"

response = get_response(question, openai=True)

display(Markdown(response))

# Authority to Require Maintenance of Financial Records
Yes, competent authorities or resolution authorities have the power to require an institution to maintain detailed records of financial contracts (BRRD_EURLEX.pdf, page 110). This requirement is intended to ensure that the authorities can fulfill their responsibilities effectively.

Source:
- From `BRRD_EURLEX.pdf`, page 110 of the European Union Finance Laws, from the Official Journal of the European Union.

## **Comparing Hugging Face and OpenAI Responses**
By comparing the **Hugging Face embeddings response** and the **OpenAI embeddings response**, we can observe that both methods retrieve relevant chunks of legal information but cite different sections of the document. Both responses emphasize the authorities' ability to enforce record-keeping, with slight differences in phrasing and the referenced pages of the document.
